<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

In [4]:
with open('login.txt') as f: # the login data is saved in .txt file for security reasons
    for line in f:
        exec(line)

In [37]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [9]:
# текст запроса
query = f''' SELECT 
                COUNT(id) 
            FROM vacancies
'''

df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [10]:
query = f''' SELECT 
                COUNT(DISTINCT id) 
            FROM employers
'''

df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [13]:
query = f''' SELECT 
                COUNT(DISTINCT name) 
            FROM areas
'''

df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [14]:
query = f''' SELECT 
                COUNT(DISTINCT name) 
            FROM industries
'''

df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,294


***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [38]:
query = f''' SELECT 
                COUNT(*) as cnt,
                areas.name
            FROM 
                vacancies 
            JOIN areas ON vacancies.area_id = areas.id
            GROUP BY areas.name
            ORDER BY cnt DESC
            LIMIT 5
            
'''

df = pd.read_sql_query(query , connection)
df


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [42]:
query = f''' SELECT 
                COUNT(*) as cnt
                
            FROM 
                vacancies 
            WHERE salary_from  IS NOT NULL OR salary_to IS NOT NULL
'''

df = pd.read_sql_query(query , connection)
df


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [47]:
query = f''' SELECT 
                ROUND(AVG(salary_from)) as mean_salary_from,
                ROUND(AVG(salary_to)) as mean_salary_to             
            FROM 
                vacancies 
            WHERE salary_from  IS NOT NULL OR salary_to IS NOT NULL
'''

df = pd.read_sql_query(query , connection)
df


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,mean_salary_from,mean_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [54]:
query = f''' SELECT 
                COUNT(*) as cnt,
                schedule, 
                employment      
            FROM 
                vacancies 
            GROUP BY schedule, employment
            ORDER BY cnt DESC
            LIMIT 2
'''

df = pd.read_sql_query(query , connection)
df


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [58]:
query = f''' SELECT 
                COUNT(*) as cnt,
                experience    
            FROM 
                vacancies 
            GROUP BY experience
            ORDER BY cnt 

'''

df = pd.read_sql_query(query , connection)
df


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

# Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [60]:
query = f'''SELECT 
                COUNT(employers.name) AS cnt,
                areas.name
            FROM 
                employers
            JOIN areas ON areas.id = employers.area
            GROUP BY areas.name
'''
employers_in_area = pd.read_sql_query(query , connection)
print(employers_in_area.shape)
employers_in_area.head(2)


(654, 2)


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt,name
0,1,Пыть-Ях
1,2,Бодайбо


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [61]:
query = f''' (SELECT 
                COUNT (*) AS cnt, 
                areas.name
            FROM 
                vacancies
            JOIN areas ON vacancies.area_id = areas.id
            GROUP BY  areas.name  
            )

'''

df = pd.read_sql_query(query , connection)
print(df.shape)

df.head(2)

(769, 2)


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt,name
0,1,Анна
1,5,Пыть-Ях


In [62]:
areas_in_emp =  employers_in_area.name.values.tolist()

employers_without_vacancies = list(set(areas_in_emp)  - set(df.name.values.tolist()))

dicts = {}
for i in employers_without_vacancies:
    for j in range(employers_in_area.shape[0]):
        if employers_in_area['name'][j] == i:
            dicts[i] = employers_in_area['cnt'][j]
            
max(dicts, key=dicts.get) 


'Россия'

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [79]:
query = f'''SELECT 
                COUNT(DISTINCT vacancies.area_id) AS cnt,
                employers.name
            FROM 
                vacancies
            JOIN employers ON employers.id = vacancies.employer_id
            GROUP BY  employers.name
            ORDER BY cnt DESC
            
'''
df = pd.read_sql_query(query , connection)
df.head(2)

,id
0,4898037
1,2056231


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [81]:
query = f'''SELECT 
                id 
            FROM 
                employers
            
            EXCEPT
            
            SELECT 
                employer_id 
            FROM 
                employers_industries
            
'''
df = pd.read_sql_query(query , connection)
df.shape

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


(8419, 1)

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [96]:
query = f'''SELECT 
                COUNT(employers_industries.employer_id) as cnt,
                employers.name
            FROM 
                employers_industries
            JOIN employers ON employers.id = employers_industries.employer_id
            GROUP BY employers.name
            HAVING COUNT(employers_industries.employer_id)=4
            ORDER BY employers.name
            LIMIT 4        
'''
df = pd.read_sql_query(query , connection)
df

,cnt,name
0,4,101 Интернет
1,4,21vek.by
2,4,2ГИС
3,4,2К


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [113]:
query = f'''SELECT 
                COUNT(*) as cnt,
                industries.name
            FROM 
                employers_industries
            JOIN industries ON industries.id = employers_industries.industry_id
            WHERE industries.name LIKE 'Разработка программного обеспечения'
            GROUP BY industries.name     
'''
df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt,name
0,3553,Разработка программного обеспечения


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [172]:
import requests
from bs4 import BeautifulSoup
url ='https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', class_= "standard sortable")

df = pd.read_html(str(table))
df = pd.concat(df)
million_cities  = df['Город'].str.split('[', expand=True)[0].tolist()
million_cities


['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [150]:
query = f'''(SELECT 
                areas.name as name,
                COUNT (*) AS cnt
            FROM 
                vacancies
            JOIN areas ON vacancies.area_id = areas.id
            JOIN employers ON employers.id  = vacancies.employer_id
            WHERE employers.name LIKE 'Яндекс'
            AND areas.name in ('Москва',  'Санкт-Петербург',  'Новосибирск',
                                'Екатеринбург',  'Казань',  'Нижний Новгород',
                                'Челябинск',  'Красноярск',
                                'Самара',  'Уфа',  'Ростов-на-Дону',
                                'Омск',  'Краснодар',  'Воронеж',
                                'Пермь',  'Волгоград')
            GROUP BY  areas.name  )
            
            UNION
            
            (SELECT 
                'total',
                COUNT(*) AS cnt
            FROM 
                vacancies
            JOIN areas ON vacancies.area_id = areas.id
            JOIN employers ON employers.id  = vacancies.employer_id
            WHERE employers.name LIKE 'Яндекс'
            AND areas.name in ('Москва',  'Санкт-Петербург',  'Новосибирск',
                                'Екатеринбург',  'Казань',  'Нижний Новгород',
                                'Челябинск',  'Красноярск',
                                'Самара',  'Уфа',  'Ростов-на-Дону',
                                'Омск',  'Краснодар',  'Воронеж',
                                'Пермь',  'Волгоград'))
                                
            ORDER BY cnt
            

'''


***

In [151]:

df = pd.read_sql_query(query , connection)
print(df.shape)
df


(17, 2)


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


# Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [147]:
query = f'''SELECT 
                COUNT(name) as cnt
            FROM 
                vacancies
            WHERE LOWER(name) LIKE '%data%' 
                OR LOWER(name) LIKE '%данн%'
'''
df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
1. 'data scientist'
2. 'data science'
3. 'исследователь данных'
4. 'ML' (здесь не нужно брать вакансии по HTML)
5. 'machine learning'
6. 'машинн%обучен%'

**В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [152]:
query = f'''SELECT 
                COUNT(*) as cnt
            FROM 
                vacancies
            WHERE (LOWER(name) LIKE '%data scientist%' 
                OR LOWER(name) LIKE '%data science%'
                OR name LIKE '%ML%'
                OR LOWER(name) LIKE '%исследователь данных%'
                OR LOWER(name) LIKE '%machine learning%'
                OR LOWER(name) LIKE '%машинн%обучен%')
                AND 
                (LOWER(name) LIKE '%junior%'
                OR experience = 'Нет опыта'
                OR employment = 'Стажировка')
                AND 
                (LOWER(name) NOT LIKE '%html%')
                
'''
df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

**Критерии для отнесения вакансии к DS указаны в предыдщуем задании.*

In [149]:
query = f'''SELECT 
                COUNT(*) as cnt
            FROM 
                vacancies
            WHERE (LOWER(name) LIKE '%data scientist%' 
                OR LOWER(name) LIKE '%data science%'
                OR name LIKE '%ML%'
                OR LOWER(name) LIKE '%исследователь данных%'
                OR LOWER(name) LIKE '%machine learning%'
                OR LOWER(name) LIKE '%машинн%обучен%')
                AND 
                (LOWER(name) NOT LIKE '%html%')
                AND 
                (key_skills LIKE '%SQL%'
                OR key_skills LIKE '%postgres%')

                
'''
df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

**Это можно сделать помощью запроса, аналогичного предыдущему.*

In [153]:
query = f'''SELECT 
                COUNT(*) as cnt
            FROM 
                vacancies
            WHERE (LOWER(name) LIKE '%data scientist%' 
                OR LOWER(name) LIKE '%data science%'
                OR name LIKE '%ML%'
                OR LOWER(name) LIKE '%исследователь данных%'
                OR LOWER(name) LIKE '%machine learning%'
                OR LOWER(name) LIKE '%машинн%обучен%')
                AND 
                (LOWER(name) NOT LIKE '%html%')
                AND 
                (LOWER(key_skills) LIKE '%python%')

                
'''
df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [142]:
query = f'''SELECT 
                AVG(array_length(regexp_split_to_array(key_skills, CHR9), 1))
            FROM 
                vacancies
            WHERE (LOWER(name) LIKE '%data scientist%' 
                OR LOWER(name) LIKE '%data science%'
                OR name LIKE '%ML%'
                OR LOWER(name) LIKE '%исследователь данных%'
                OR LOWER(name) LIKE '%machine learning%'
                OR LOWER(name) LIKE '%машинн%обучен%')
                AND 
                (LOWER(name) NOT LIKE '%html%')
                AND 
                (key_skills IS NOT NULL)
                
'''
df = pd.read_sql_query(query , connection)
df.round(2)

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,avg
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем **указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [34]:
query = f'''
            SELECT 
                experience,
                AVG((salary_from)) as salary, 
                COUNT(*) as cnt
            FROM 
                vacancies
            WHERE (salary_from IS NOT NULL
                    AND salary_to IS NULL)
            
                AND ((LOWER(name) LIKE '%data scientist%' 
                OR LOWER(name) LIKE '%data science%'
                OR name LIKE '%ML%'
                OR LOWER(name) LIKE '%исследователь данных%'
                OR LOWER(name) LIKE '%machine learning%'
                OR LOWER(name) LIKE '%машинн%обучен%')
                AND 
                (LOWER(name) NOT LIKE '%html%'))
            GROUP BY experience
                            
            UNION 
            
            SELECT 
                experience,
                AVG(salary_to) as salary,
                COUNT(*) as cnt
            FROM 
                vacancies
            WHERE (salary_to IS NOT NULL
                    AND salary_from IS NULL)
            
                AND ((LOWER(name) LIKE '%data scientist%' 
                OR LOWER(name) LIKE '%data science%'
                OR name LIKE '%ML%'
                OR LOWER(name) LIKE '%исследователь данных%'
                OR LOWER(name) LIKE '%machine learning%'
                OR LOWER(name) LIKE '%машинн%обучен%')
                AND 
                (LOWER(name) NOT LIKE '%html%'))
                
            GROUP BY experience
                
            UNION 
            
            SELECT 
                experience,
                AVG((salary_from + salary_to)/2) as salary,
                COUNT(*) as cnt
            FROM 
                vacancies
            WHERE ( salary_from IS NOT NULL
                AND salary_to IS NOT NULL)
                AND ((LOWER(name) LIKE '%data scientist%' 
                OR LOWER(name) LIKE '%data science%'
                OR name LIKE '%ML%'
                OR LOWER(name) LIKE '%исследователь данных%'
                OR LOWER(name) LIKE '%machine learning%'
                OR LOWER(name) LIKE '%машинн%обучен%')
                AND 
                (LOWER(name) NOT LIKE '%html%'))
                
                GROUP BY experience
            
'''
df = pd.read_sql_query(query , connection)


c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,experience,salary,cnt
0,Нет опыта,60000.000000,2
1,От 3 до 6 лет,248829.636364,11
2,Нет опыта,89166.666667,3
3,От 3 до 6 лет,250996.250000,8
4,От 1 года до 3 лет,130833.333333,6
5,От 3 до 6 лет,227375.000000,8
6,Нет опыта,67500.000000,2
7,От 1 года до 3 лет,106918.250000,8
8,От 1 года до 3 лет,162181.928571,14


***

In [35]:
df['weighted_salary'] = df['salary'] * df['cnt'] / df.groupby(['experience'])['cnt'].transform('sum')
df.groupby(['experience']).sum().round()

,salary,cnt,weighted_salary
experience,,,
Нет опыта,216667.0,7,74643.0
От 1 года до 3 лет,399934.0,28,139675.0
От 3 до 6 лет,727201.0,27,243115.0


# Общий вывод по проекту

In [45]:
query = f'''SELECT 
                COUNT(*) as cnt,
                employers.name
            FROM 
                vacancies 
            JOIN employers ON vacancies.employer_id = employers.id
            GROUP BY employers.name   
            ORDER BY cnt DESC
            LIMIT 5
'''
df = pd.read_sql_query(query , connection)
df

c:\Users\vi04wecu\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


In [36]:
connection.close()

На рынке находится около 50 тысяч открытых вакансий от 23т работодателей. Больше всего вакансих предлагают гиганты рынка, например Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нейть. Это говорит либо о расширении компаний, либо о большой текучке кадров. 

Для большинства трбуется от 1 до 3 лет опыта работы (около 25 тыс.), далее по популярности от 3 до 6 лет. Наимение популярными являются специалисты с более 6 годами опыта. Таким образом, слежнее всего на рынке труда приходится самым молодым и самым опытным специалистам. Можно сделать вывод, что работодатели ищут специалистов с опыт, но не слишком дорогих. В базе представленны вакансии из 1300 населенных пунктов. Географически, вакансии сконцетрированы в столицах и крупных городах, а именно в Москве, СПб, Минске, Новосибирске. 
Болтшинство работодателей не побликуют информацию о заработной плате, а те, кто предоставляют такие сведения, в среднем предлогают от 71 до 110 т.р.. 
Больше всего вакансий требуют полной занятосни с полным рабочем днем или удаленной работой.


